Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue

import numpy as np

In [2]:
PROBLEM_SIZE = 10
NUM_SETS = 20
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)])
    for _ in range(NUM_SETS)
)

#SETS = sorted(SETS, reverse=True, key=lambda e : sum(e))

State = namedtuple('State', ['taken', 'not_taken'])

In [3]:
def goal_check(state):
    return np.all(reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    ))


def coverage(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)])
    )


def distance(state):
    p = reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )

    return PROBLEM_SIZE - sum(p)


def compute_min_tiles(state):   #look how many blocks are still not filled and estimate the minimum number of tiles required to fill them
    covered = coverage(state)
    missing = PROBLEM_SIZE - sum(covered)
    sort_cand = sorted(sum(np.logical_and(SETS[c], np.logical_not(covered)) for c in state.not_taken), reverse=True)
    taken = 1
    while sum(sort_cand[:taken]) < missing:
        taken += 1
    return taken


def compute_min_tiles1(state):  #like the previous one but it actually stops when the whole set is covered
    covered = coverage(state)
    missing = PROBLEM_SIZE - sum(covered)
    sort_cand = sorted(state.not_taken, key=lambda c : sum(np.logical_and(SETS[c], np.logical_not(covered))), reverse=True)
    taken = 0
    for c in sort_cand:
        if np.all(covered):
            break
        covered = np.logical_or(covered, SETS[c])
        taken += 1
    return taken


def compute_min_tiles2(state):   #this actually computes how many tiles are required to fill the whole space, but it's not an estimation
    added = 0
    covered = coverage(state)
    while not np.all(covered):
        # missing = PROBLEM_SIZE - sum(covered)
        best_cand = max(state.not_taken, key=lambda c : sum(np.logical_and(SETS[c], np.logical_not(covered))))
        state = State(
            state.taken ^ {best_cand},
            state.not_taken ^ {best_cand},
        )
        added += 1
        covered = np.logical_or(covered, SETS[best_cand])
    return added


def compute_min_tiles3(state):  #sort the remaining tiles based on the amount of non overlapping blocks
    added = 0
    covered = coverage(state)
    sort_cand = sorted(state.not_taken, key=lambda c : sum(np.logical_and(SETS[c], covered)))
    for c in sort_cand:
        if np.all(covered):
            break
        covered = np.logical_or(covered, SETS[c])
        added += 1
    return added


def compute_min_tiles4(state): #similar to the previous one, the idea is to select the tiles that are filling non-filled blocks and are not filling already filled blocks
    added = 0
    covered = coverage(state)
    sort_cand = sorted(state.not_taken, key=lambda c : sum(np.logical_xor(SETS[c], covered)), reverse=True)
    for c in sort_cand:
        if np.all(covered):
            break
        covered = np.logical_or(covered, SETS[c])
        added += 1
    return added

def compute_min_tiles5(state):  #sorts for number of overlapping blocks and actual number of false blocks of tiles
    added = 0
    covered = coverage(state)
    sort_cand = sorted(state.not_taken, key=lambda c : sum(np.logical_and(SETS[c], covered)) + sum(np.logical_not(SETS[c])))
    for c in sort_cand:
        if np.all(covered):
            break
        covered = np.logical_or(covered, SETS[c])
        added += 1
    return added


In [4]:
assert goal_check(
    State(set(range(NUM_SETS)), set())
), "Problem not solvable"

In [5]:
frontier = PriorityQueue()
already_explored = []
opt_sol = None
found_solutions = []
# frontier = SimpleQueue()

state = State(set(), set(range(NUM_SETS)))
frontier.put((distance(state), state))

counter = 0
skip_count = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    if sorted(current_state.taken) not in already_explored:
        already_explored.append(sorted(current_state.taken))
        counter += 1
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action},
                current_state.not_taken ^ {action},
            )

            frontier.put((len(new_state.taken) + compute_min_tiles4(new_state), new_state))

    else:
        skip_count += 1
    _, current_state = frontier.get()


print(
    f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)"
)

Solved in 9 steps (3 tiles)


In [6]:
print(current_state.taken)

{1, 18, 13}


In [7]:
print(skip_count)

0


In [8]:
print(len(already_explored))

9


In [10]:
for i in current_state.taken:
    print(SETS[i])

[False  True False False  True  True  True False False False]
[False False False  True False  True False False  True  True]
[ True False  True  True False False False  True False False]
